In [65]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
'''
Calculate count, FPKM, and FPKM-UQ values.
FPKM and FPKM-UQ were defined here:
https://docs.gdc.cancer.gov/Data/Bioinformatics_Pipelines/Expression_mRNA_Pipeline/#upper-quartile-fpkm
'''

import sys
import os
import shutil
import subprocess
import numpy as np
from optparse import OptionParser
from time import strftime

__author__ = "Nitish Mishra"
__maintainer__ = "Nitish Mishra"
__email__ = "nitish.mishra@stjude.org"


def printlog (mesg):
	'''print progress into stderr and log file'''
	mesg="@ " + strftime("%Y-%m-%d %H:%M:%S") + ": " + mesg
	print(mesg, file=sys.stderr)


def cal_fpkm(count_file, infor_file, out_file,log2_flag=False):
	'''
	parameters
	----------
	count_file : str
		count file generated by HT-Seq. The first column is gene ID, the second
		column is read count.
	infor_file : str
		Information file.
			gene_id gene_name       seqname start   end     strand  gene_type       gene_status     havana_gene     full_length     exon_length     exon_num
			ENSG00000223972.5       DDX11L1 chr1    11869   14409   +       transcribed_unprocessed_pseudogene      KNOWN   OTTHUMG00000000961.2    2541    1735    9
			ENSG00000238009.5       RP11-34P13.7    chr1    89295   133723  -       lincRNA NOVEL   OTTHUMG00000001096.2    44429   3726    17
			ENSG00000230415.1       RP5-902P8.10    chr1    1275223 1280420 +       lincRNA NOVEL   OTTHUMG00000002234.2    5198    513     5
	'''

	printlog ('Read gene information file: %s' % infor_file)
	gene_sizes = {} # mRNA size for all genes
	gene_infor = {}
	protein_coding = set()	#list of protein coding genes
	for l in open(infor_file):
		l = l.strip()
		if l.startswith('gene_id'):continue
		f = l.split()
		gene_sizes[f[0]] = int(f[11])
		gene_infor[f[0]] = '\t'.join(f[1:6])
		if f[6] == 'protein_coding':
			protein_coding.add(f[0])

	print ('\tTotal genes: %d' % len(gene_sizes), file=sys.stderr)
	print ('\tTotal protein-coding genes: %d' % len(protein_coding), file=sys.stderr)


	printlog('Read gene count file to calculate 75 percentile count and total count: %s' % count_file)
	gene_counts = []
	for l in open(count_file):
		l = l.strip()
		if l.startswith('__'):
			continue
		f = l.split()
		gene_id = f[0]
		if gene_id not in protein_coding:
			continue
		gene_counts.append(int(f[1]))

	uq_count = np.percentile(sorted(gene_counts), 75)
	total_count = sum(gene_counts)
	print ('\tTotal protein-coding genes: %d' % len(gene_counts), file=sys.stderr)
	print ('\tThe 75 perentile count of protein-coding genes: %f' % (uq_count), file=sys.stderr)
	print ('\tThe total count of protein-coding genes: %f' % (total_count), file=sys.stderr)

	FPKM_OUT = open(out_file, 'w')
	if log2_flag  is True:
		print ('\t'.join(['gene_ID','gene_name','chrom','start','end','strand', 'raw_count', 'FPKM(log2(x+1))', 'FPKM-UQ(log2(x+1))']), file=FPKM_OUT)
	else:
		print ('\t'.join(['gene_ID', 'gene_name','chrom','start','end','strand', 'raw_count', 'FPKM', 'FPKM-UQ']), file=FPKM_OUT)
	print ('Read gene count file to calculate FPKM and FPKM-UQ: %s' % count_file, file=sys.stderr)
	for l in open(count_file):
		l = l.strip()
		if l.startswith('__'):
			continue
		f = l.split()
		gene = f[0]
		count = int(f[1])
		if gene in gene_sizes:
			try:
				if log2_flag  is True:
					fpkm_uq = np.log2((count*1000000000)/(gene_sizes[gene]*uq_count) +1)
					fpkm = np.log2((count*1000000000)/(gene_sizes[gene]*total_count) +1)
				else:
					fpkm_uq = (count*1000000000)/(gene_sizes[gene]*uq_count)
					fpkm = (count*1000000000)/(gene_sizes[gene]*total_count)

			except:
				fpkm_uq = 'NA'
				fpkm = 'NA'

		else:
			fpkm_uq = 'NA'
			fpkm = 'NA'
		print (gene + '\t' + gene_infor[gene] + '\t' + '\t'.join([str(i) for i in (count, fpkm, fpkm_uq)]), file=FPKM_OUT)

	FPKM_OUT.close()


In [66]:
cal_fpkm(count_file = "htseq-annot.genes.results.txt", infor_file = "geneinfo.ensembl.GRCm39.104.rdna.txt", out_file = "Htseq.FPKM", log2_flag=False)

@ 2022-03-03 15:14:39: Read gene information file: geneinfo.ensembl.GRCm39.104.rdna.txt
	Total genes: 55426
	Total protein-coding genes: 21885
@ 2022-03-03 15:14:39: Read gene count file to calculate 75 percentile count and total count: htseq-annot.genes.results.txt
	Total protein-coding genes: 21885
	The 75 perentile count of protein-coding genes: 193.000000
	The total count of protein-coding genes: 5355015.000000
Read gene count file to calculate FPKM and FPKM-UQ: htseq-annot.genes.results.txt
